#**Final submission-SARIMA forecast**

In [ ]:
# It is recommended to upgrade the statsmodels library. 
# Uncomment the below code to upgrade statsmodels
!pip install statsmodels --upgrade

In [ ]:
import pandas as pd
import warnings
import itertools
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
#to ignore warnings
import warnings
import itertools
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('MER_T12_06.xlsx')
df.head()

In [ ]:
#conversion of "YYYYMM" columnn into standard datetime format & making it as index
# We are using errors=’coerce’. It will replace all non-numeric values with NaN.

dateparse = lambda x: pd.to_datetime(x, format='%Y%m', errors = 'coerce')
df = pd.read_excel('MER_T12_06.xlsx', parse_dates=['YYYYMM'], index_col='YYYYMM', date_parser=dateparse) 
df.head()

**The arguments can be explained as:**

- **parse_dates:** This is a key to identify the date time column. Example, the column name is ‘YYYYMM’.
- **index_col:** This is a key that forces pandas to use the date time column as index.
- **date_parser:** Converts an input string into datetime variable.

# Exploratory Data Analysis

- Let us first identify and **drop the non datetimeindex** rows. First, let's convert the index to datetime, coerce errors, and filter NaT

In [ ]:
ts = df[pd.Series(pd.to_datetime(df.index, errors='coerce')).notnull().values]
ts.head()
ts.describe().T

**Observations**
1. The observations have reduced to 4707 after filtering on NaT
2. There are 9 unique categories in MSN and Description columns
3. The 'Value' coulmn has missing values with a high frequency of 384. The rows with these missing values should be eliminated

In [ ]:
#convert the emision value into numeric value
nat=pd.DataFrame(pd.to_numeric(ts['Value'],errors='coerce')).convert_dtypes()
ts['Value']=nat['Value']

In [ ]:
#Drop the missing value using dropna(inplace = True)
ts.dropna(inplace = True)
ts.describe().T

In [ ]:
ts.dtypes

### **Natural gas based CO2 emission forecasting**

For developing the time series model and forecasting, you are expected to use the natural gas CO2 emission from the electrical power generation. We need to slice this data:

In [ ]:
###Slice the data to get the monthly total CO2 emissions of Natural Gas Electric Power Sector
natural=ts[ts['MSN']=='NNEIEUS']
natural= pd.DataFrame(natural).drop(['Description','MSN'],axis=1)

In [ ]:
 #Check 1st few rows of data
natural.head()

# **Split the dataset**

In [ ]:
# Split the data into train and test
# using first 41 years data as training data
train_data = natural.loc['1973-01-01':'2014-01-01']

# using the last 2 years data as test data
test_data = natural.loc['2014-01-01':'2016-07-01']
print(train_data)
print(test_data)

###**Test the Stationarity**

In [ ]:
#Import the required package

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import matplotlib.pyplot as plt

###**Test the stationarity through Visualization**

In [ ]:
# creating a subplot space
fig, ax = plt.subplots(figsize=(16, 6))

# plotting train data
train_data.plot(ax=ax)

# plotting test data
test_data.plot(ax=ax)

# adding the legends in sequential order
plt.legend(['train data', 'test data'])

# showing the time which divides the original data into train and test
plt.axvline(x='2014-01-01', color='black', linestyle='--')

# showing the plot
plt.show()
fig.savefig("series", bbox_inches='tight', dpi=600)

In [ ]:
# Calculate the rolling mean and standard deviation for a window of 12 observations
rolmean=train_data.rolling(window=12).mean()
rolstd=train_data.rolling(window=12).std()

# Visualize the rolling mean and standard deviation
fig2=plt.figure(figsize=(16,8))
actual = plt.plot(train_data, color='cyan', label='Actual Series')
rollingmean = plt.plot(rolmean, color='red', label='Rolling Mean') 
rollingstd = plt.plot(rolstd, color='green', label='Rolling Std. Dev.')
plt.title('Rolling Mean & Standard Deviation of the Series')
plt.legend()
plt.show()
fig2.savefig("Rolling_mean", bbox_inches='tight', dpi=600)

#### **Observations and Insights: ____**
1. Series has upward trend, it is not stationary
2. The rolling mean does a good job in capturing the avergae trend of the data. However, it fails to capture seasonal patterns in the data.

# **Test the stationarity using the Augmented Dickey-Fuller Test**

Use the **Augmented Dickey-Fuller (ADF) Test** to verify if the series is stationary or not. The null and alternate hypotheses for the ADF Test are defined as:

**- Null hypothesis:** The Time Series is non-stationary


**- Alternative hypothesis:** The Time Series is stationary

In [ ]:
#Define a function to use adfuller test
def adfuller(train_data):
  #Importing adfuller using statsmodels
  from statsmodels.tsa.stattools import adfuller
  print('Dickey-Fuller Test: ')
  adftest = adfuller(train_data['Value'])
  adfoutput = pd.Series(adftest[0:4], index=['Test Statistic','p-value','Lags Used','No. of Observations'])
  for key,value in adftest[4].items():
    adfoutput['Critical Value (%s)'%key] = value
  print(adfoutput)
adfuller(train_data)

- **Observations and Insights**
- 1. From the above test, we can see that the p-value = 0.995 i.e. > 0.05 (For 95% confidence intervals) therefore, we fail to reject the null hypothesis.
- 2. Hence, we can confirm that the series is non-stationary.

# **Transformation of the dataset into a stationary one**

**We can use some of the following methods to convert a non-stationary series into a stationary one:**


1. Log Transformation
2. Differencing the series 

We take the average of ‘k’ consecutive values depending on the frequency of time series (in this capstone 12 months). 

Here, we will take the average over the past 1 year.

In [ ]:
 # Visualize the rolling mean and standard deviation after using log transformation
plt.figure(figsize=(16,8))
df_log = np.log(train_data) #Reduced variance
MAvg = df_log.rolling(window=12).mean()
MStd = df_log.rolling(window=12).std()
fig3=plt.plot(df_log)
plt.plot(MAvg, color='r', label = 'Moving Average')
plt.plot(MStd, color='g', label = 'Standard Deviation')
plt.legend()
plt.show()

**Observations and Insights: _____**
- Since **we can still see the upward trend in the series**, we can conclude that **the series is still non-stationary.** 
- However, the standard deviation is almost constant which implies that **now the series has constant variance.**

**Visualize the rolling mean and rolling standard deviation of the shifted series (df_shift) and check the stationarity by calling the adfuller() function. Also, write your observations on the same.**

In [ ]:
plt.figure(figsize=(16,8))
df_shift = df_log - df_log.shift(periods = 1,fill_value=0)
MAvg_shift = df_shift.rolling(window=12).mean()
MStd_shift = df_shift.rolling(window=12).std()
plt.plot(df_shift, color='c')
plt.plot(MAvg_shift, color='red', label = 'Moving Average')
plt.plot(MStd_shift, color='green', label = 'Standard Deviation')
plt.legend()
plt.show()

#Dropping the null values that we get after applying differencing method
df_shift = df_shift.dropna()

**Observations and Insights:___**
- 1.Since we can no longer see an upward trend, the series seems to be almost constant (stationary)

- 2. The standard deviation also seems to be almost constant

**Lets verify using Augmented Dickey-Fuller (ADF) Test**

In [ ]:
adfuller(df_shift)

**Observations and Insights:**
1. From the above test, we can see that the p-value = 1.93e-8 i.e. < 0.05 (For 95% confidence intervals) therefore, we can reject the null hypothesis.
2. Hence, we can confirm that the series is now stationary.



**Obtaining stationarity through seasonal differencing**

In [ ]:
from statsmodels.tsa.stattools import adfuller
# implementing ADF test on the original time series data
result = adfuller(train_data['Value'])

fig, ax = plt.subplots(figsize=(16, 6))
train_data.plot(ax=ax)
plt.show()

# printing the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
# taking seasonal differencing of the timeseries
train_data_stationary = train_data.diff(periods=12).dropna()

# implementing ADF test on the first order differenced time series data
result = adfuller(train_data_stationary['Value'])

fig, ax = plt.subplots(figsize=(16, 6))
train_data_stationary.plot(ax=ax)
plt.show()

# printing the results
print('ADF Statistic:', result[0])
print('p-value:', result[1])

#### **Observations and Insights: _____**
1. p-value=2.44e-6<0.05. Stationarity is obtained by 1st order differencing. Therefore D=1 in SARIMA modelling

Let's decompose the time series to check its different components.

### **Elimination of trend and seasonality: Decomposition**

In [ ]:
#Importing the seasonal_decompose function to decompose the time series

from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(train_data)

trend = decomp.trend
seasonal = decomp.seasonal
residual = decomp.resid

plt.figure(figsize=(15,10))
plt.subplot(411)
plt.plot(train_data, label='Actual', marker='.')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend, label='Trend', marker='.')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal, label='Seasonality', marker='.')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(residual, label='Residuals', marker='.')
plt.legend(loc='upper left')
plt.tight_layout()

**Observations and Insights: ____**
- We can see that there are significant **trend, seasonality and residuals components** in the series
- The plot for seasonality shows that **Natural gas based CO2 emissions spike in July and August.**

**Now let's move on to the model building section. First, we will plot the `ACF` and `PACF` plots to get the values of p and q i.e. order of AR and MA models to be used.**

**Plot the auto-correlation function and partial auto-correlation function to get p and q values for AR, MA, ARMA, and ARIMA models**

### **Find optimal parameters (p,d,q)x(P,D,Q) and build the SARIMA model**

**Plot the ACF and PACF charts and find the optimal parameters**

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

plt.figure(figsize = (16,8))
plot_acf(df_shift, lags = 24) 
plt.show() 
plot_pacf(df_shift, lags = 24) 
plt.show()

**Observations and Insights: _____**

**Observations:**
- From the above PACF plot we can see that **the lag** at which the plot extends beyond the statistically significant boundary for the first time is **lag 1.** 
- This indicates that **lag 1 (p=1)** should be sufficient to fit the data.
- We observed a periodicity on the PACF since the underlying data is periodic with a period of 12 months. Hence, 'P' which is the order of seasonal AR term in SARIMA model is equal to 1.

- Similarly, from the ACF plot, we can infer that **q=1.**
- The ACF and PACF also capture the seasonality in the data
- We observed a periodicity on the ACF since the underlying data is periodic with a period of 12 months. Hence, 'Q' which is the order of seasonal MA term in SARIMA model is equal to 1.

In [ ]:
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

# SARIMA Model

To Fit and predict the shifted series with the SARIMA Model and calculate the RMSE

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 3
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA: ')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
# Determing p,d,q combinations with AIC scores.
for param in pdq:
    for param_seasonal in seasonal_pdq:
        mod = sm.tsa.statespace.SARIMAX(df_log.astype(float),
                                        order=param,
                                        seasonal_order=param_seasonal,
                                        enforce_stationarity=False,enforce_invertibility=False)

        results = mod.fit(disp=0)

        print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

**Observations**:
1. The (p,d,q)x(P,D,Q) combination that resulted in the least AIC score (-1048.0736) is (1,1,1)x(1,0,1,12). Therefore, this combination is a good starting point for fitting the SARIMA model on the training data.

**Fitting SARIMA model**

In [ ]:
#creating an SARIMA model with parameters (1, 1, 1)x(1, 0, 1, 12)
mod = sm.tsa.statespace.SARIMAX(df_log.astype(float),
                                order=(1, 1, 1),
                                seasonal_order=(1, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit(disp=0)

print(round(results.aic,2))
print(results.summary().tables[1])
results.plot_diagnostics(figsize=(15, 12))
plt.show()

**Observations:**

Top left: The residual errors seem to fluctuate around a mean of zero.

Top Right: The density plot suggests that the distribution of residuals is very close to a standard normal distribution.

Bottom left: The quantile-quantile plot indicates that the sample quantiles are approximately equal to the theoritical quantiles. Any significant deviations would imply the distribution of residuals is skewed.

Bottom Right: The ACF plot shows the residuals are not autocorrelated as no lag other than 0 is significant. Any autocorrelation would imply that there is some pattern in the residual errors which are not explained in the model.

The above plots show that the model assumptions are not violated.

**Inverse Transformation**

**Use the correct inverse transformation depending on the model chosen to get back the original values**

**Apply an inverse transformation on the predictions of the chosen model**

In [ ]:
# Printing the fitted values
predictions=pd.Series(results.fittedvalues)
#predictions

In [ ]:
#Third step - applying exponential transformation
predictions_SARIMA = np.exp(predictions)#use exponential function
#predictions_SARIMA

**Plot the original vs predicted series of the training data**

In [ ]:
#Code here
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(train_data, color = 'c', label = 'Original Series')  #plot the original train series
plt.plot(predictions_SARIMA, color = 'r', label = 'Predicted Series')  #plot the predictions_ARIMA 
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

**Observations & Insights**:  
- We can see that the predicted series is very similar to the original series i.e. The model is good at predicting values on the training data, apart from one outlier prediction in 1974.**
- The seasonal aspect of the SARIMA model helps us better capture the seasonal variations in the data.

# Forecast from 2014 to 2016 to check if the predictions align with the test data

In [ ]:
#Add the code blocks based on the requirements
forecasted_SARIMA = results.forecast(steps=31)#forecast using the results_ARIMA for next 31 months. Keep steps=24
forecasted_SARIMA

In [ ]:
list2 = forecasted_SARIMA.tolist()
series2 = pd.Series(list2)
#series2

In [ ]:
index = pd.date_range('2013-12-01','2016-07-01' , freq='1M')
df2 = pd.DataFrame()
df2['forecasted'] = np.exp(series2)
df2.index = index
#df2

**Plot the original vs predicted series**

In [ ]:
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(train_data, color = 'c', label = 'Training data')
plt.plot(predictions_SARIMA, color = 'r', label = 'Prediction on Train data') #plot the predictions_ARIMA series
plt.plot(test_data,color='b',label='Test Data')
plt.plot(df2, label = 'Forecast', color='g')  #plot the forecasted_ARIMA series
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

**Let's test the RMSE of the transformed predictions and the original value on the training and testing data to check whether the model is giving a generalized performance or not**

In [ ]:
from sklearn.metrics import mean_squared_error
error =np.sqrt(mean_squared_error(predictions_SARIMA, train_data)) #calculate RMSE using the predictions_ARIMA and df_train 
error

In [ ]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(forecasted_SARIMA, test_data))
error#calculate RMSE using the forecasted_ARIMA and df_test

**Observations and Insights**
1. The above plot shows that the model is able to identify the trend and seasonalities in the data. 
2. The forecasted values are similar to the test data
3. Based on the RMSE values and the plot, it looks like this model is able to capture the trend and seasonal patterns well

**Observations**:
1. The small RMSE values also indicate that this is a good model to forecast CO2 emission values

# Forecast for the next 24 months from original data

In [ ]:
#creating an SARIMA model with parameters (1, 1, 1)x(1, 0, 1, 12)
df_log_original=np.log(natural)
mod_original = sm.tsa.statespace.SARIMAX(df_log_original.astype(float),
                                order=(1, 1, 1),
                                seasonal_order=(1, 0, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results_original = mod_original.fit(disp=0)

print(round(results_original.aic,2))
print(results_original.summary().tables[1])
results_original.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
# Printing the fitted values
predictions_final=pd.Series(results_original.fittedvalues)
#predictions

In [ ]:
#Third step - applying exponential transformation
predictions_SARIMA_final = np.exp(predictions_final)#use exponential function
#predictions_SARIMA

In [ ]:
#Add the code blocks based on the requirements
forecasted_SARIMA_final = results_original.forecast(steps=48)#forecast using the results_ARIMA for next 24 months. Keep steps=24
#forecasted_SARIMA

In [ ]:
list2 = forecasted_SARIMA_final.tolist()
series2 = pd.Series(list2)
#series2

In [ ]:
index = pd.date_range('2016-07-02','2020-07-02' , freq='1M')- pd.offsets.MonthBegin(1)
df3 = pd.DataFrame()
df3['forecasted'] = np.exp(series2)
df3.index = index
#df2

In [ ]:
#Plotting the original vs predicted series
plt.figure(figsize=(16,8))
plt.plot(natural, color = 'c', label = 'Original Series')
plt.plot(df3, color = 'b', label = 'Forecast') #plot the predictions_ARIMA series
plt.title('Original vs Forecast')
plt.legend()
plt.show()

**Executive Summary**
1. Upon trying various models such as AR,MA, ARMA and ARIMA to fit the natural gas CO2 emissions data from 1973 to 2016, SARIMA model seems to fit the data very well.
2. This model captures the trend as well as the seasonalities in the dtraining data and and fits well with the test data too
3. The natural gas CO2 emissions forecast fot the next 24 months (2016 to 2018) indicates an upwards trend ans seasonliities eith spikes in July and August every year 

**Problem and Solution Summary**
- 1. The problem in question is to understand how the CO2 emissions due to natural gasconsumption would behave in the future.
- 2. This would potentially help us in designing new energy efficient policies.
- 3. Key solution design points:
    - a. CO2 emissions due to natural gas consumptions in the last 40 years, displays trends and seasonal patterns.The forecasted data should be able to predict such trends and seasonal occurances.
    - b. SARIMA model is able to capture trend and seasonalities. Hence, it is chosen for fitting the time series daata of CO2 emissions due to natural gas consumption
    - The data was split into training data (1973-2014) and test data (2014-2016)
    - c. The SARIMA model was tuned using grid search on the training data, and the metric used to select the best model was AIC. The best model was ARIMA(1,1,1)x(1,0,1,12), AIC: -1048
    - d. Further, the performance of the model was evaluated by plotting the predicted values and comparing to that of the test data using RMSE
    - f. Finally, the forecast for the next 48 months (2016-2020) was obtained using the chosen model. The forecast indicates upward trends and seasonalities with periodic spikes in July and AUgust of every year. 

**Recommendations for Implementation**
- 1. Co2 emissions maybe influenced by numerous other features in addition to using lagged values. Hence, a supplemental analysis done by identifying with stakeholders key drivers and improving model forecasts would be an immediate next step
- 2. Forecasts may be used to assess environmental/health costs and used as a lever to justify increased investments in renewable energy like solar, wind etc
- 3. A key risk in implementation is regional/geographical variation in Co2 behaviour. This study may need to be done acroos multiple geographies to justify investment in renewable sources
- 4. A cost/benefit analysis maybe done using the forecasts. What is the cost of reducing natural gas CO2 emissions? Would the benefit gained by shifting to other sources outweigh the short term cost of shifting sources?